# Common filters
 
Uses filters that are commonly used in Paraview such as gradient calculations and also uses the raw data and the Numpy package to compute the $\lambda_2$ vortex identification criterion which is given by where the second largest eigenvalue of the matrix 
$$S_{ik}S_{kj} + \Omega_{ik}\Omega_{kj}$$
is negative where 

$$S_{ij} = \frac{1}{2}\left(\frac{\partial u_{i}}{\partial x_j} + \frac{\partial u_{j}}{\partial x_i}\right)$$
$$\Omega_{ij} = \frac{1}{2}\left(\frac{\partial u_{i}}{\partial x_j} - \frac{\partial u_{j}}{\partial x_i}\right)$$



In [1]:
import pyvista as pv
import numpy as np
fn = 'case/postprocessing/RESULTS_FLUID_DOMAIN.case'

reader = pv.get_reader(fn)
reader.set_active_time_value(300)

data = reader.read()['Fluid domain']

print(data)

UnstructuredGrid (0x76a3d1fb3880)
  N Cells:    3140920
  N Points:   3172091
  X Bounds:   -1.000e+00, 1.000e+00
  Y Bounds:   -1.000e+00, 1.000e+00
  Z Bounds:   0.000e+00, 2.000e+01
  N Arrays:   10


In [2]:
# Q criterion
# Can use qcriterion=True or qcriterion='key', which results in a data set of that name
Q = data.compute_derivative('Velocity',qcriterion='Q', gradient=False)
print(Q.cell_data)

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)
    Q                       float32    (3140920,)


In [3]:
# Contour like the contour filter in paraview with Q as the selected scalar
Q_contour = Q.ctp().contour([0.5],
                      scalars='Q')

In [4]:
# Plot isosurface of Q
p = pv.Plotter(window_size=(500,500))

p.add_mesh(Q_contour,
           scalars='Velocity',
           cmap='bwr')

p.add_axes()

# set camera
p.view_xy(negative=True)
# change azimuth and elevation of the camera
p.camera.azimuth = 30
p.camera.elevation = 30
p.show()

Widget(value='<iframe src="http://localhost:40771/index.html?ui=P_0x76a3d1fdf610_0&reconnect=auto" class="pyvi…

In [5]:
# We now compute lambda_2 from the gradient calculation

dudx_data = data.compute_derivative('Velocity',gradient='dudx')
# dudx is of size (n_cell, 9) as we should expect
print(dudx_data.cell_data)

pyvista DataSetAttributes
Association     : CELL
Active Scalars  : mpi_rank_id
Active Vectors  : Velocity
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (3140920,)           SCALARS
    Velocity                float32    (3140920, 3)         VECTORS
    Pressure                float32    (3140920,)
    TurbVisc                float32    (3140920,)
    CourantNb               float32    (3140920,)
    FourierNb               float32    (3140920,)
    total_pressure          float32    (3140920,)
    U_mean                  float32    (3140920, 3)
    UU_mean                 float32    (3140920, 6)
    P_mean                  float32    (3140920,)
    dudx                    float32    (3140920, 9)


In [6]:
# Reshape data so last dimensions are a 3X3 matrix
dudx = dudx_data.cell_data['dudx'].reshape((dudx_data.n_cells ,3,3))

# transpose last two dimentions of dudx
dudxT = np.transpose(dudx,axes=(0,2,1))

# compute strain rate tensor and rotation rate tensor
S = 0.5*(dudx + dudxT)
Omega = 0.5*(dudx - dudxT)

In [7]:
#Compute $S_{ik}S_{kj} + \Omega_{ik}S_{kj}$
S2O2 = np.matmul(S,S) + np.matmul(Omega,Omega)

# Compute eigenvalues of symmetric matrix leaving unused eigenvectors blank
eig, _ = np.linalg.eigh(S2O2)

# Get second largest value
lambda2 = eig[:,1]

# Reassign lambda_2 to cell_data
dudx_data.cell_data['lambda_2'] = lambda2

In [8]:
# Plot contour the same manner as Q
lambda2_contour = dudx_data.ctp().contour([-0.4],
                                    scalars='lambda_2')

p = pv.Plotter(window_size=(500,500))

p.add_mesh(lambda2_contour,
           scalars='Velocity',
           cmap='bwr')

p.add_axes()
p.view_xy(negative=True)
p.camera.azimuth = 30
p.camera.elevation = 30
p.show()


Widget(value='<iframe src="http://localhost:40771/index.html?ui=P_0x76a3a6f3bf90_1&reconnect=auto" class="pyvi…